## Samhitha Gundam
## HW 2 - part 2
**Question 5a, 5b present in this notebook and Q 1,2,3,4 is present in the other Notebook**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim.models
import gensim.downloader as api
import nltk
import numpy as np
nltk.download('punkt')
import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler

[nltk_data] Downloading package punkt to /Users/samhitha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
raw_data=pd.read_csv("amazon_reviews_us_Jewelry_v1_00.tsv",sep="\t",on_bad_lines='skip')
data=raw_data[['star_rating','review_body']]
data=data.dropna()
data = data.reset_index(drop=True)
data['star_rating']=data['star_rating'].astype(int)

/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3250/4051224240.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data=pd.read_csv("amazon_reviews_us_Jewelry_v1_00.tsv",sep="\t",on_bad_lines='skip')


In [4]:
df=data.groupby('star_rating').sample(n=20000)

df['review_body']=df['review_body'].str.lower()  #convert to lower case

#remove contractions
df['review_body']=df['review_body'].str.replace("\'re"," are")
df['review_body']=df['review_body'].str.replace("br"," ")
df['review_body']=df['review_body'].str.replace("\n't"," not")
df['review_body']=df['review_body'].str.replace("\'s'"," is")
df['review_body']=df['review_body'].str.replace("i'm"," i am")
df['review_body']=df['review_body'].str.replace("\'ve'"," have")
df['review_body']=df['review_body'].str.replace("din't","did not")

df['review_body']=df['review_body'].str.replace('http\S+|www.\S+', '', case=False)
df['review_body']=df['review_body'].str.replace('[^a-zA-Z0-9 ]', '')
df['review_body']=df['review_body'].str.replace('/ +/', ' ')#convert multispace to space
df['review_body']=df['review_body'].str.replace('/^ /', '') # remove spaces in the start of the string
df['review_body']=df['review_body'].str.replace('/ $/', '') # remove unnecesary space at the end of the string


/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3250/343968516.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_body']=df['review_body'].str.replace('http\S+|www.\S+', '', case=False)
/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3250/343968516.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_body']=df['review_body'].str.replace('[^a-zA-Z0-9 ]', '')
/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3250/343968516.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_body']=df['review_body'].str.replace('/ +/', ' ')#convert multispace to space
/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3250/343968516.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_body']=df['review_body'].str.replace('/^

In [5]:
wv = api.load('word2vec-google-news-300')

In [6]:
X=df['review_body']
Y=df['star_rating']

X=pd.DataFrame(X)
sentences=[k.split() for k in X['review_body']]
X['reviews_split']=sentences

In [7]:
features=[]
for lis in X['reviews_split']:
    wordvecs=[]
    vector=np.zeros(300,)
    for j in range(len(lis)):
        if(j>=20):
            break
        else:
            if lis[j] in wv.key_to_index:
                wordvecs.append(wv[lis[j]])
    while len(wordvecs)<20:
        wordvecs.append(np.zeros(300,))
        
    features.append(wordvecs)
    
X['input_3']=features

In [8]:
X

,review_body,reviews_split,input_3
911218,made carelessly the second i put it on my wris...,"[made, carelessly, the, second, i, put, it, on...","[[-0.055908203, 0.11767578, 0.2109375, 0.00836..."
668073,the pair they gave me looks nothing like the o...,"[the, pair, they, gave, me, looks, nothing, li...","[[0.080078125, 0.10498047, 0.049804688, 0.0534..."
680243,color peels off i wished i would have read the...,"[color, peels, off, i, wished, i, would, have,...","[[-0.0043029785, 0.14355469, 0.036376953, 0.12..."
317872,elastic string doesnt work fluent afraid of e...,"[elastic, string, doesnt, work, fluent, afraid...","[[0.17285156, -0.084472656, -0.29101562, 0.185..."
1048382,very upset the ring when i got it has black st...,"[very, upset, the, ring, when, i, got, it, has...","[[0.016601562, 0.045654297, -0.119140625, 0.06..."
...,...,...,...
1197729,ive always hated wearing necklaces because the...,"[ive, always, hated, wearing, necklaces, becau...","[[-0.41210938, 0.18847656, -0.234375, 0.296875..."
960962,i am so pleased with this purchase i am pregn...,"[i, am, so, pleased, with, this, purchase, i, ...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
1311381,this earcuff was small simple and wellmade ser...,"[this, earcuff, was, small, simple, and, wellm...","[[0.109375, 0.140625, -0.03173828, 0.16601562,..."
99933,i love it it fits well,"[i, love, it, it, fits, well]","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."


In [9]:
df = X.input_3.apply(pd.Series)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
911218,"[-0.055908203, 0.11767578, 0.2109375, 0.008361...","[0.40429688, -0.008666992, 0.11230469, 0.09179...","[0.080078125, 0.10498047, 0.049804688, 0.05346...","[0.13378906, -0.024414062, 0.07128906, 0.07568...","[-0.22558594, -0.01953125, 0.09082031, 0.23730...","[-0.014221191, 0.10058594, 0.00034332275, 0.06...","[0.084472656, -0.0003528595, 0.053222656, 0.09...","[0.026733398, -0.09082031, 0.027832031, 0.2041...","[0.14453125, 0.04711914, 0.10058594, 0.328125,...","[0.007019043, 0.0087890625, -0.33203125, -0.12...","[0.084472656, -0.0003528595, 0.053222656, 0.09...","[0.22753906, 0.29492188, -0.050048828, -0.0028...","[0.080078125, 0.10498047, 0.049804688, 0.05346...","[0.17285156, -0.084472656, -0.29101562, 0.1855...","[-0.1171875, 0.078125, -0.2890625, -0.02661132...","[0.19921875, -0.080566406, -0.21191406, 0.2001...","[-0.25976562, 0.27148438, 0.119628906, 0.00723...","[-0.21679688, 0.1171875, 0.036621094, 0.007812...","[-0.068359375, 0.15039062, 0.034423828, -0.138...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
668073,"[0.080078125, 0.10498047, 0.049804688, 0.05346...","[-0.09863281, -0.087402344, -0.26367188, 0.053...","[0.064453125, 0.036132812, 0.03857422, 0.09472...","[0.22265625, -0.016601562, 0.13476562, -0.1337...","[0.13867188, -0.091796875, 0.03491211, 0.15039...","[0.024047852, 0.31445312, -0.026245117, 0.0717...","[0.1640625, 0.056396484, 0.08251953, -0.065917...","[0.103515625, 0.13769531, -0.0029754639, 0.181...","[0.080078125, 0.10498047, 0.049804688, 0.05346...","[0.045654297, -0.14550781, 0.15625, 0.16601562...","[-0.22558594, -0.01953125, 0.09082031, 0.23730...","[0.16699219, -0.05419922, -0.087402344, 0.0196...","[-0.047607422, 0.08154297, 0.045654297, 0.0917...","[0.01159668, 0.21679688, 0.14746094, 0.1010742...","[0.23242188, 0.020996094, 0.028686523, 0.05126...","[0.009338379, 0.032958984, -0.32421875, 0.1650...","[-0.22558594, -0.01953125, 0.09082031, 0.23730...","[0.103515625, 0.13769531, -0.0029754639, 0.181...","[0.080078125, 0.10498047, 0.049804688, 0.05346...","[-0.09863281, -0.087402344, -0.26367188, 0.053..."
680243,"[-0.0043029785, 0.14355469, 0.036376953, 0.129...","[-0.033935547, 0.16015625, 0.12060547, 0.06738...","[-0.071777344, -0.11035156, 0.008239746, 0.151...","[-0.22558594, -0.01953125, 0.09082031, 0.23730...","[-0.03149414, 0.26367188, 0.100097656, 0.00842...","[-0.22558594, -0.01953125, 0.09082031, 0.23730...","[0.08935547, 0.12988281, 0.21289062, 0.1777343...","[-0.13964844, -0.03466797, -0.053710938, 0.179...","[-0.03173828, -0.10986328, -0.11669922, 0.1777...","[0.080078125, 0.10498047, 0.049804688, 0.05346...","[-0.27929688, 0.030029297, -0.18164062, -0.077...","[0.122558594, -0.08935547, 0.026977539, 0.0737...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
317872,"[0.17285156, -0.084472656, -0.29101562, 0.1855...","[-0.042236328, 0.080078125, -0.18652344, -0.07...","[-0.075683594, 0.033691406, -0.064941406, 0.13...","[0.012512207, -0.04736328, -0.13574219, -0.015...","[0.31054688, 0.06640625, 0.21386719, 0.0625, -...","[0.17871094, 0.091308594, -0.0016555786, 0.177...","[-0.15039062, 0.072265625, 0.030517578, 0.0415...","[0.080078125, 0.10498047, 0.049804688, 0.05346...","[0.06689453, -0.1796875, 0.018676758, 0.141601...","[0.08203125, -0.07910156, 0.05053711, -0.02392...","[0.16992188, 0.049072266, 0.08154297, 0.120117...","[0.0070495605, -0.07324219, 0.171875, 0.022583...","[-0.22558594, -0.01953125, 0.09082031, 0.23730...","[-0.16699219, -0.06640625, 0.057373047, -0.059...","[0.027832031, 0.25585938, 0.15820312, -0.04809...","[-0.011779785, -0.04

In [10]:
X_train,X_test,Y_train, Y_test = train_test_split(df,Y, test_size=0.2, random_state=30)

x=X_train
y=Y_train
test_x=X_test
test_y=Y_test

In [11]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, output_size):
        super(RNN, self).__init__()

        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        
        self.i2h1 = nn.Linear(input_size + hidden_size2, hidden_size1)
        self.h1h2 = nn.Linear(hidden_size1, hidden_size2)
        self.h2o = nn.Linear(hidden_size2, output_size)
        
        #self.i2o = nn.Linear(input_size + hidden_size2, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat([input,hidden],1)
        hidden1 = self.i2h1(combined)
        hidden2 = self.h1h2(hidden1)
        output = self.h2o(hidden2)
        output = self.softmax(output)
        return output, hidden2

    def initHidden1(self):
        return torch.zeros(1,self.hidden_size1)
    
    def initHidden2(self):
        return torch.zeros(1, self.hidden_size2)
    
rnn = RNN(300, 50,10, 5)
learning_rate=0.1
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs=1

rnn =rnn.float()
for i in range(epochs):
    
    for j in range(len(x)):
        #print("j",j)
        hidden2 = rnn.initHidden2()
        optimizer.zero_grad()
        #pred=[]
        #Hidden state - 20. Sending word by word vectors into RNN
        for index in range(20):
            temp=x.iloc[j,index]
            temp=np.reshape(temp,(1,300))
            temp_df=torch.tensor(temp)
            output, hidden2 = rnn(temp_df.float(), hidden2.float())
        #print("output",output)
        if(j==0):
            pred=output
        else:
            pred=torch.cat([pred,output],0)
        
    loss = criterion(pred, torch.tensor(y.to_numpy()-1))
    loss.backward()

    optimizer.step()
    print("Epoch ",i ," done")

In [ ]:
for j in range(len(test_x)):
    hidden2 = rnn.initHidden2()
    optimizer.zero_grad()
    #pred=[]
    for index in range(20):
        temp=test_x.iloc[j,index]
        temp=np.reshape(temp,(1,300))
        temp_df=torch.tensor(temp)
        output, hidden2 = rnn(temp_df.float(), hidden2.float())
    #pred.append(output)
    if(j==0):
        pred=output
    else:
        pred=torch.cat([pred,output],0)

_,predicted = torch.max(pred.data,1)
correct = (predicted == test_y).sum()
print("Accuracy RNN 5(a):",correct,"/",len(test_y))

**Question 5b**

In [ ]:
# define the GRU architecture
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super().__init__()

    # Number of hidden dimensions
    self.hidden_dim = hidden_dim

    # Number of hidden layers
    self.layer_dim = layer_dim

    # GRU
    self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True)
    # Output layer
    self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

    # Initialize hidden state with zeros
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)

    # One time step
    out, hn = self.gru(x, h0)
    out = self.fc(out[:, -1, :])
    return out

gru = GRUModel(300, 20, 1, 5)

criterion = nn.CrossEntropyLoss()

optimizer= torch.optim.SGD(gru.parameters(), lr=0.01)

In [ ]:
epochs=1

gru =gru.float()
for i in range(epochs):
    
    for j in range(len(x)):

        optimizer.zero_grad()
        pred=[]
        for index in range(20):
            temp=x.iloc[j,index]
            temp=np.reshape(temp,(1,300))
            temp_df=torch.tensor(temp)
            output = gru(temp_df.float())
        pred.append(output)
    #print(output.shape)
    ten=pred[0]
    for t in range(1,len(pred)):
        torch.cat([ten,t],0)
    
    print(ten.shape())
    loss = criterion(output, torch.tensor(y))
    loss.backward()

    optimizer.step()

In [ ]:
for j in range(len(test_x)):
    hidden2 = rnn.initHidden2()
    optimizer.zero_grad()
    #pred=[]
    for index in range(20):
        temp=test_x.iloc[j,index]
        temp=np.reshape(temp,(1,300))
        temp_df=torch.tensor(temp)
        output, hidden2 = rnn(temp_df.float(), hidden2.float())
    #pred.append(output)
    if(j==0):
        pred=output
    else:
        pred=torch.cat([pred,output],0)

_,predicted = torch.max(pred.data,1)
correct = (predicted == test_y).sum()
print(correct/len(test_y))